In [29]:
from __future__ import print_function
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
x = torch.empty(5, 3)
y = torch.empty(5, 3)
y.fill_(10)
print(x,'\n',y)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.9562e-42, 0.0000e+00],
        [0.0000e+00, 7.1746e-43, 0.0000e+00]]) 
 tensor([[10., 10., 10.],
        [10., 10., 10.],
        [10., 10., 10.],
        [10., 10., 10.],
        [10., 10., 10.]])


<h1>How to offload to GPU</h1>

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    y = torch.ones_like(x, device=device)
    x = x.to(device)
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))

C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\__init__.py:117: UserWarning: 
    Found GPU0 Quadro K1100M which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


RuntimeError: cuda runtime error (48) : no kernel image is available for execution on the device at c:\a\w\1\s\tmp_conda_3.6_104352\conda\conda-bld\pytorch_1550400396997\work\aten\src\thc\generic/THCTensorMath.cu:14

<h1>AUTOGRAD: Automatic Differentation</h1>

In [4]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [5]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [6]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward1>)


In [7]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


<h2>Gradients</h2>

In [8]:
print(x.grad)

None


<h1>Neural Networks</h1>

In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        #self.conv1 = nn.Conv2d(1, 6, 32)

        # 6 input image channels, 16 output channels, 5x5 square convolution
        # kernel
        #self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv2 = nn.Conv2d(6, 32, 5)
        
        # fc is Full Connection or Forward Channel..
        #self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc1 = nn.Linear(32, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
            return num_features
        
net = Net()
print(net)
    

        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=32, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)


In [22]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [31]:
TRAINING_ROUNDS = 10

input = torch.randn(1, 1, 32, 32)
output = net(input)

# calculate the loss function
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(out, target)

# update the weights with the new gradient
# simple way: weight = weight - learning_rate * gradient
'''
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
'''
# better way: using an optimizer package that can implement functions such as
# SGD, Nesterov-SGD, Adam, RMSProp, etc.
optimizer = optim.SGD(net.parameters(), lr=0.01)

for i in range(TRAINING_ROUNDS - 1):
    optimizer.zero_grad()    # zeroes the gradient buffers
    output = net(input)
    loss = criterion(output, target)
    loss.backward()     # backpropagate
    optimizer.step()    # does the update each round